In [1]:
import numpy as np
import pandas as pd
from bqplot import (
    ColorScale, DateScale, LinearScale, Bars, Lines, Axis, Figure, CATEGORY10
)
import random

rows_to_keep = 43

sheet_data = pd.read_excel("https://docs.google.com/spreadsheets/d/1DuYUj2ODS8D3PWK42ZopUD1dqcg89ckI6vPn71LidGo/export?format=xlsx")
sheet_data = sheet_data.iloc[:rows_to_keep].drop(columns=["Overall % Lost", "Imgur"]).dropna(axis=1, how="all")
sheet_data = sheet_data.rename(columns={"$": "Pot Contribution", "Starting 1/1/22": "Starting Weight"})
sheet_data = sheet_data.melt(id_vars=["Participant", "Pot Contribution", "Paid?", "Starting Weight"], var_name="Date", value_name="Weight")
sheet_data["Starting Weight"] = pd.to_numeric(sheet_data["Starting Weight"])
sheet_data["Weight"] = pd.to_numeric(sheet_data["Weight"])
sheet_data = sheet_data.dropna(subset=["Weight", "Starting Weight"]).sort_values(by=["Participant", "Date"]).reset_index()

by_participant = sheet_data.groupby("Participant")

sheet_data["Previous Week Weight"] = by_participant["Weight"].shift(1)
sheet_data.loc[sheet_data["Previous Week Weight"].isnull(), "Previous Week Weight"] = sheet_data.loc[sheet_data["Previous Week Weight"].isnull(), "Starting Weight"]
sheet_data["Weight Difference"] = sheet_data["Weight"] - sheet_data["Previous Week Weight"]

sheet_data["Cumulative Weight Lost"] = by_participant["Weight Difference"].transform(pd.Series.cumsum)
sheet_data["Participant Index"] = by_participant.ngroup()

participant_colors = ["#%06x" % random.randint(0, 0xFFFFFF) for participant, _ in by_participant]

display(sheet_data)


,index,Participant,Pot Contribution,Paid?,Starting Weight,Date,Weight,Previous Week Weight,Weight Difference,Cumulative Weight Lost,Participant Index
0,30,A_Weino,0.0,NaN,247.0,2022-01-01,247.0,247.0,0.0,0.0,0
1,73,A_Weino,0.0,NaN,247.0,2022-01-07,246.2,247.0,-0.8,-0.8,0
2,116,A_Weino,0.0,NaN,247.0,2022-01-14,241.8,246.2,-4.4,-5.2,0
3,159,A_Weino,0.0,NaN,247.0,2022-01-21,241.8,241.8,0.0,-5.2,0
4,128,Chrisg42,0.0,NaN,202.1,2022-01-14,202.1,202.1,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
145,142,tejastacostand - Goal 140,0.0,NaN,153.2,2022-01-21,151.2,148.2,3.0,-2.0,36
146,0,uncomfortablyhigh,50.0,Yes,269.0,2022-01-01,269.0,269.0,0.0,0.0,37
147,43,uncomfortablyhigh,50.0,Yes,269.0,2022-01-07,253.6,269.0,-15.4,-15.4,37
148,86,uncomfortablyhigh,50.0,Yes,269.0,2022-01-14,252.4,253.6,-1.2,-16.6,37


In [2]:
participant_lines = []


x_date_scale = DateScale()
y_linear_scale = LinearScale()
color_scale = ColorScale(colors=participant_colors)

x_axis = Axis(scale=x_date_scale, grid_lines='solid', label='Date')
y_axis = Axis(scale=y_linear_scale, orientation='vertical', tick_format='0.2f', grid_lines='solid', label='Weight')

for participant, participant_data in by_participant:
    participant_lines.append(
        Lines(
            name=participant, 
            x=participant_data["Date"],
            y=participant_data["Weight"],
            scales={'x': x_date_scale, 'y': y_linear_scale, "color": color_scale}, 
            labels = participant_data["Participant"].values.tolist(), 
            display_legend=True,
            color=participant_data["Participant Index"]
        )
    )
# test_line = Lines(
#         x=sheet_data["Date"],
#         y=sheet_data["Weight"],
#         scales={'x': x_date_scale, 'y': y_linear_scale, "color": color_scale}, 
#         # labels=sheet_data["Participant"].values.tolist(), 
#         display_legend=True,
#         color=sheet_data["Participant Index"]
#         )


# fig = Figure(marks=[test_line], axes=[x_axis, y_axis], title='Weight over time', legend_location='top-right')

fig = Figure(marks=participant_lines, axes=[x_axis, y_axis], title='Weight over time', legend_location='top-right')

display(fig)

Figure(axes=[Axis(label='Date', scale=DateScale()), Axis(label='Weight', orientation='vertical', scale=LinearS…

In [3]:
import plotly.express as px
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()